In [5]:
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error

In [6]:
data = pd.read_csv('/home/tanmaypatil/Documents/DS/hybrid-recommendation-system/data/movies_grouplens/ratings.dat',
                   delimiter='::', header=None, engine='python', encoding='latin1')
data.columns = ['user', 'event', 'rating', 'timestamp']

# Create a user-item matrix
ratings_matrix = data.pivot(
    index='user', columns='event', values='rating').fillna(0)

In [3]:
# Perform SVD
U, sigma, Vt = np.linalg.svd(ratings_matrix)

k = 1000
U = U[:, :k]
sigma = np.diag(sigma[:k])
Vt = Vt[:k, :]

In [7]:
U_1, sigma_1, Vt_1 = np.linalg.svd(ratings_matrix)

k_2 = 96
U_1 = U_1[:, :k_2]
sigma_1 = np.diag(sigma_1[:k_2])
Vt_1 = Vt_1[:k_2, :]

In [8]:
u_1_shape = list(U_1.shape)
sig_1_shape = list(sigma_1.shape)
v_1_shape = list(Vt_1.shape)

In [9]:
paramters_1 = u_1_shape[0] * u_1_shape[1] + \
    sig_1_shape[0] + v_1_shape[0] * v_1_shape[1]
paramters_1

935712

In [26]:
u_shape = list(U.shape)
sig_shape = list(sigma.shape)
v_shape = list(Vt.shape)

In [27]:
paramters = u_shape[0] * u_shape[1] + sig_shape[0] + v_shape[0] * v_shape[1]
paramters

9747000

In [10]:
# Reconstruct the original matrix
predicted_ratings = np.dot(np.dot(U_1, sigma_1), Vt_1)

# Convert to DataFrame
predicted_ratings_df = pd.DataFrame(
    predicted_ratings, columns=ratings_matrix.columns, index=ratings_matrix.index)

In [11]:
# Calculate RMSE
def calculate_rmse(actual, predicted):
    # Keep only non-zero elements to match actual ratings
    nonzero_indexes = np.nonzero(actual)
    actual_nonzero = actual[nonzero_indexes]
    predicted_nonzero = predicted[nonzero_indexes]

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(actual_nonzero, predicted_nonzero))
    return rmse

In [12]:
actual_ratings_matrix = ratings_matrix.values
rmse = calculate_rmse(actual_ratings_matrix, predicted_ratings)

print("RMSE:", rmse)

# Recommendation for a specific user (e.g., user with ID 22)
user_id = 22
user_ratings = predicted_ratings_df.loc[user_id]
top_recommendations = user_ratings.sort_values(ascending=False).head(10)

print("Top recommendations for user", user_id)
print(top_recommendations)

RMSE: 2.078302834229093
Top recommendations for user 22
event
608     5.543310
2804    5.145380
2997    4.887406
589     4.874319
1240    4.771125
1259    4.541953
2791    4.378053
1265    4.302413
318     4.284745
3578    4.261340
Name: 22, dtype: float64
